In [27]:
# import libraries
import plotly.plotly as py
import plotly.graph_objs as go
import colorlover as cl
from IPython.display import HTML
import scipy
import math
import csv
import pandas as pd
import numpy as np

master=pd.read_excel('C:/Users/liuz2/Documents/Projects/SuicideRates/WHO_all.xlsx', \
    sheet_name='Sheet1', usecols='A:AM')    
master=master.rename(columns={'Countries':'country'})
# Remove rows that we don't have population or GDP data on
master=master[~master['country'].isin(['Rodrigues','Reunion','Occupied Palestinian Territory',\
    'Netherlands Antilles','Montserrat','Mayotte','Martinique','Anguilla','TFYR Macedonia',\
    'Saint Pierre and Miquelon','Falkland Islands (Malvinas)','French Guiana','Guadeloupe',\
    'Total reporting countries'])]
master['country']=np.where(master['country']=='Bahamas', 'Bahamas, The', 
    np.where(master['country']=='Egypt', 'Egypt, Arab Rep.',
    np.where(master['country']=='Hong Kong SAR', 'Hong Kong SAR, China',
    np.where(master['country']=='Iran (Islamic Rep of)', 'Iran, Islamic Rep.',
    np.where(master['country']=='Kyrgyzstan', 'Kyrgyz Republic', 
    np.where(master['country']=='Macau', 'Macao SAR, China', 
    np.where(master['country']=='Republic of Moldova','Moldova',
    np.where(master['country']=='Republic of Korea', 'Korea, Rep.',
    np.where(master['country']=='Saint Kitts and Nevis', 'St. Kitts and Nevis',
    np.where(master['country']=='Saint Lucia', 'St. Lucia',
    np.where(master['country']=='Saint Vincent and Grenadines', 'St. Vincent and the Grenadines',
    np.where(master['country']=='Slovakia','Slovak Republic', 
    np.where(master['country']=='United States of America','United States',
    np.where(master['country']=='Venezuela (Bolivarian Republic of)','Venezuela, RB',
    np.where(master['country']=='Virgin Islands (USA)','Virgin Islands (U.S.)',   
    master['country'])))))))))))))))
# Transpose into long format
master=pd.melt(master, id_vars=['country'], var_name='year', value_name='suicides_no')
master['year']=master['year'].astype('int64')
master['suicides_no'] = pd.to_numeric(master['suicides_no'],errors='coerce')

uem=pd.read_csv('C:/Users/liuz2/Documents/Projects/SuicideRates/UEM.csv')
uem=uem[uem['Indicator Code']=='SL.UEM.TOTL.NE.ZS']
country=uem[['Country Name', 'Country Code']]
country.drop_duplicates(keep='first',inplace=True)
country.columns=['country','iso3c']
not_country=["ARB","CEB", "CSS","EAP","EAR","EAS","ECA","ECS","EMU","EUU","FCS","HIC","HPC","IBD","IBT","IDA","IDB",
"IDX","INX","LAC","LCN","LDC","LIC","LMC","LMY","LTE","MEA","MIC","MNA","NAC","OED","OSS","PRE","PSS","PST","SAS","SSA","SSF","SST","TEA","TEC","TLA","TMN","TSA","TSS","UMC","WLD"]
country=country.assign(not_country=lambda x: x['iso3c'].isin(not_country))

def makelong(dat,varname):
    output_dat=dat.drop(['Indicator Name','Indicator Code','Country Name'],axis=1)
    output_dat=pd.melt(output_dat, id_vars=['Country Code'], var_name='year', value_name=varname)
    output_dat.columns=['iso3c','year',varname]
    return(output_dat)
uem=makelong(uem, 'UEM_TOTL')
uem['year']=uem['year'].astype('int64')

master=pd.merge(master, country, how='left', on=['country'])
master=pd.merge(master, uem, how='left', left_on=['iso3c','year'], right_on=['iso3c','year'])

cdat=pd.read_csv("C:/Users/liuz2/Documents/Projects/World Bank/whatawaste/country_level_data_2.csv", encoding="utf-8")
region=cdat[['region_id','country_name','income_id']]
regions= {
    'ECS': 'Europe and Central Asia',
    'SSF': 'Sub-Saharan Africa',
    'LCN': 'Latin America and the Caribbean',
    'EAS': 'East Asia and Pacific',
    'MEA': 'Middle East and North Africa',
    'SAS': 'South Asia',
    'NAC': 'North America', 
}
region['region']=region['region_id'].map(regions)
incomes={
    'HIC': 'High-income',
    'UMC': 'Upper-middle income',
    'LMC': 'Lower-middle income',
    'LIC': 'Low-income'
}
region['income']=region['income_id'].map(incomes)
region.columns=['region_id','country','income_id','region','income']
master=pd.merge(master, region, how='left',left_on=['country'], right_on=['country'])
del cdat

master['region']=np.where(master['iso3c']=='STP', 'Sub-Saharan Africa', master['region'])
master['region_id']=np.where(master['iso3c']=='STP', 'SSF', master['region_id'])

popu=pd.read_csv('C:/Users/liuz2/Documents/Projects/World Bank/Population/API_SP.POP.TOTL_DS2_en_csv_v2_10576638.csv', encoding="utf-8", skiprows=4)
popu=popu.drop(['Unnamed: 63','2018','Indicator Name','Indicator Code','Country Name'],axis=1)
populong=pd.melt(popu, id_vars=['Country Code'], var_name='year', value_name='pop')
populong.columns=['iso3c','year','population']
populong['year']=populong['year'].astype('int64')
master=pd.merge(master, populong, how='left', left_on=['iso3c','year'], right_on=['iso3c','year'])
del popu

gdp=pd.read_csv('C:/Users/liuz2/Documents/Projects/World Bank/GDP/GDP-Per-Capita-PPP-Constant-2011-International/API_NY.GDP.PCAP.PP.KD_DS2_en_csv_v2_41408.csv',\
     encoding='utf-8', skiprows=4, nrows=264)
gdp=gdp.drop(['Indicator Name', 'Indicator Code', 'Country Name', 'Unnamed: 63'], axis=1)
gdplong=pd.melt(gdp, id_vars=['Country Code'], var_name='year', value_name='gdp')
gdplong.columns=['iso3c','year','gdp']
gdplong.year=gdplong.year.str.slice(0,4)
gdplong['gdp']=pd.to_numeric(gdplong['gdp'], errors='coerce') 
gdplong['loggdp']=np.log(gdplong['gdp'])
gdplong['year']=gdplong['year'].astype('int64')
master=pd.merge(master,gdplong, how='left', left_on=['iso3c','year'], right_on=['iso3c','year'])
del gdp

gini=pd.read_csv('C:/Users/liuz2/Documents/Projects/World Bank/Gini/API_SI.POV.GINI_DS2_en_csv_v2_126296.csv', encoding='utf-8', nrows=264)
gini=gini.drop(['Indicator Name','Indicator Code','Country Name'], axis=1)
ginilong=pd.melt(gini, id_vars=['Country Code'], var_name='year', value_name='gini')
ginilong.columns=['iso3c','year','gini']
ginilong['gini']=pd.to_numeric(ginilong['gini'], errors='coerce') 
ginilong['year']=ginilong['year'].astype('int64')
master=pd.merge(master,ginilong, how='left', left_on=['iso3c','year'], right_on=['iso3c','year'])
del gini

whr19=pd.read_csv('C:/Users/liuz2/Documents/Projects/SuicideRates/WHR19.csv')
whr19.columns=['country','year','LifeLadder','LogGDPPC','Social_support','Healthy_life_expectancy',\
    'Freedom_to_make_life_choice','Generosity','Perceptions_of_corruption','positive_affect','negative_affect',\
        'Confidence_in_government','Democratic_quality','Delivery_quality',\
        'LifeLadder_std', 'LifeLadder_std_mean', 'gini_whr','gini_avg', 'gini_house',\
        'trust', 'trust84', 'trust93', 'trust98', 'trust04', 'trust09', 'trust14']
master=pd.merge(master, whr19, how='left', left_on=['country','year'], right_on=['country','year'])

gallup=pd.read_excel('C:/Users/liuz2/Documents/Projects/SuicideRates/Gallup_20190806.xlsx', \
    sheet_name='Access to the Internet', skiprows=7, usecols='B:C, E:F')
# Pivot the data
gallup.columns=['country','year','gender','value']
gallup=gallup.pivot_table(values='value', index=['country','year'],columns='gender')
internet=gallup.reset_index()
internet.columns=['country','year','internet','internet_f','internet_m']
del gallup

master=pd.merge(master, internet, how='left', left_on=['country','year'], right_on=['country','year'])

gallup=pd.read_excel('C:/Users/liuz2/Documents/Projects/SuicideRates/Gallup_20190806.xlsx', \
    sheet_name='Life Today', skiprows=7, usecols='B:C, E:F')
# Pivot the data by gender
gallup.columns=['country','year','gender','value']
gallup=gallup.pivot_table(values='value', index=['country','year'],columns='gender')
happy=gallup.reset_index()
happy.columns=['country','year','happy','happy_f','happy_m']
del gallup

master=pd.merge(master, happy, how='left', left_on=['country','year'], right_on=['country','year'])

master['suicides/100kpop']=master['suicides_no']/(master['population']/100000)

def make_line_trace(
    indat, #Input Data
    x, #x axis variable
    y, #y axis variable
    name='', #Name for the legend
    mode='lines+markers', #type of plot
    label='' #text label
):
    return(
    go.Scatter(
        x=indat[x],
        y=indat[y],
        name=name,
        mode=mode,
        text=label        
    )
)
SR_country=master[master['suicides_no']>=0]
SR_country=SR_country.groupby(['country','year'])[['suicides_no','population']].sum(skipna=True).reset_index()
SR_country['suicides/100kpop']=SR_country['suicides_no']/(SR_country['population']/100000)
country2region=master[['country','iso3c','region','region_id']]
country2region=country2region.drop_duplicates()
SR_country=pd.merge(SR_country, country2region, how='left', left_on='country', right_on='country')
SR_country=pd.merge(SR_country,master[['country','year','income','income_id','gdp','loggdp','UEM_TOTL','LifeLadder','gini']], how="left",on=['country','year'])
SR_country['country_year']=SR_country['country']+SR_country['year'].map(str)


C:\Python\Python37\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Python\Python37\lib\site-packages\ipykernel_launcher.py:72: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

C:\Python\Python37\lib\site-packages\ipykernel_launcher.py:79: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



# Suicide Rates

## Which countries have the higest suicide rates
 The first question we ask is which country or countries have the highest suicide rates.

In [28]:
All_ctr=list(country2region['country'])
All_ctr_trace=[]
for country in All_ctr:
    All_ctr_trace.append(make_line_trace(SR_country[SR_country['country']==country],x='year',y='suicides/100kpop',\
        name=country,mode='lines+markers',label=SR_country[SR_country['country']==country]['year']))
layout1 = dict(title = 'Suicide Rates from 1979 to 2016',
              yaxis = dict(zeroline = False,
                    title='Suicide Count per 100k',
                ),
              xaxis = dict(zeroline = False,
                    title='Year',
              )
             )
fig = go.Figure(data=All_ctr_trace, layout=layout1)
py.iplot(fig)
